In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc, col, max
from pyspark.ml.feature import StringIndexer
from pyspark.ml import pipeline
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.sql import functions as F

In [0]:
spark = SparkSession.builder.appName("Spotify_recommend").getOrCreate()

In [0]:
df_listenings = spark.read.option("header","true").option("inferschema","true").csv("/mnt/adls/test/PrachiSingh/Try/MusicRecommender")

In [0]:
display(df_listenings)

In [0]:
df_listenings = df_listenings.drop('Date')

In [0]:
df_listenings = df_listenings.na.drop()

In [0]:
# Replacing white spaces with underscore in column names
df_listenings = df_listenings.select([F.col(col).alias(col.replace(' ','_')) for col in df_listenings.columns])

In [0]:
# Converting column names to lower case
df_listenings = df_listenings.select([F.col(col).alias(col.lower()) for col in df_listenings.columns])

In [0]:
row_numbers = df_listenings.count()
col_numbers = len(df_listenings.columns)
print(row_numbers,col_numbers)

In [0]:
df_listenings_agg = df_listenings.select('user_id','track').groupby('user_id','track').agg(count('*')).withColumnRenamed("count(1)", "count") 

In [0]:
display(df_listenings_agg)

In [0]:
row_numbers = df_listenings_agg.count()
col_numbers = len(df_listenings_agg.columns)
print(row_numbers,col_numbers)

In [0]:
# Limiting our analysis with 20k records for faster computation
df_listenings_agg = df_listenings_agg.limit(20000)

In [0]:
# A label indexer that maps a string column of labels to an ML column of label indices. If the input column is numeric, we cast it to string and index the string values. The indices are in [0, numLabels). By default, this is ordered by label frequencies so the most frequent label gets index 0. The ordering behavior is controlled by setting stringOrderType. Its default value is ‘frequencyDesc’.

In [0]:
indexer = [StringIndexer(inputCol=col,outputCol=col+'_index') for col in list(set(df_listenings_agg.columns)-set(['count']))]

In [0]:
indexer_1 = StringIndexer()\
        .setInputCol('user_id')\
        .setOutputCol('user_id_indexed')\
        .setHandleInvalid('keep')

In [0]:
indexer_2 = StringIndexer()\
        .setInputCol('track')\
        .setOutputCol('track_indexed')\
        .setHandleInvalid('keep')

In [0]:
model1 = indexer_1.fit(df_listenings_agg)

In [0]:
df1= model1.transform(df_listenings_agg)

In [0]:
model2 = indexer_2.fit(df1)

In [0]:
df2 = model2.transform(df1)

In [0]:
data = df2.select('count','user_id_indexed','track_indexed')

In [0]:
# Index model using pipeline
user_id_indexer = StringIndexer(inputCol="user_id", outputCol="user_idIndex")
track_indexer = StringIndexer(inputCol="track", outputCol="trackIndex")
​pipeline = Pipeline(stages=[user_id_indexer, track_indexer])
model = pipeline.fit(df_listenings_agg).transform(df_listenings_agg)
model.show()

In [0]:
(training,test) = data.randomSplit([0.5,0.5])

In [0]:
USERID = "user_id_indexed"
TRACK = "track_indexed"
COUNT = "count"

In [0]:
# Alternating Least Squares (ALS) matrix factorization.

# ALS attempts to estimate the ratings matrix R as the product of two lower-rank matrices, X and Y, i.e. X * Yt = R. Typically these approximations are called ‘factor’ matrices. The general approach is iterative. During each iteration, one of the factor matrices is held constant, while the other is solved for using least squares. The newly-solved factor matrix is then held constant while solving for the other factor matrix.

# This is a blocked implementation of the ALS factorization algorithm that groups the two sets of factors (referred to as “users” and “products”) into blocks and reduces communication by only sending one copy of each user vector to each product block on each iteration, and only for the product blocks that need that user’s feature vector. This is achieved by pre-computing some information about the ratings matrix to determine the “out-links” of each user (which blocks of products it will contribute to) and “in-link” information for each product (which of the feature vectors it receives from each user block it will depend on). This allows us to send only an array of feature vectors between each user block and product block, and have the product block find the users’ ratings and update the products based on these messages.

In [0]:
als = ALS(maxIter=5, regParam=0.01, userCol=USERID,itemCol=TRACK, ratingCol=COUNT)
model = als.fit(training)

In [0]:
predictions = model.transform(test)

In [0]:
recs = model.recommendForAllUsers(10)

In [0]:
display(df2.filter(col('track_indexed')==758).select('track'))